# Northwind Sales Analysis — Sfirloaga Mihai-Radu

This notebook extracts order-line data from a local MySQL Northwind database (or uses prebuilt CSVs), computes KPIs, and produces visuals.

How to run:
- CSV-first: If `data/processed/order_lines.csv` exists, you can skip the DB extract. Run the KPI and plotting cells.
- Full DB: Ensure MySQL has the `northwind` schema loaded (see README), then run cells 1→3 to build the CSVs; then run the plots.


In [ ]:
%pip install -q pandas==2.2.2 matplotlib==3.9.0 mysql-connector-python==8.4.0


In [ ]:
import os
if os.path.exists("data/processed/order_lines.csv"):
    print("Found data/processed/order_lines.csv — you can skip to KPIs/plots.")
else:
    print("CSV not found — run Cell 2 (DB extract) then Cell 3 (KPIs).")


In [ ]:
import os
import pandas as pd
import mysql.connector

# Robust extract (auto-detects column names; no categories join)
# Goal: connect to MySQL, discover the exact table/column names present in your
# Northwind variant, build a resilient SELECT, and save a tidy CSV of order lines.
# Why discovery? Northwind schemas differ (e.g., order_details vs orderdetails,
# product_name vs productname). Auto-detection lets this work across variants.

# Connection settings for your local MySQL instance. Adjust if needed.
HOST = "127.0.0.1"; PORT = 3306; USER = "root"; PASSWORD = "Password"; DB = "northwind"

def cols(cur, table):
    """Return a dict {lowercased_column_name: exact_column_name} for a table.
    Helps us refer to the precise names even when variants differ.
    """
    cur.execute(f"SHOW COLUMNS FROM {table}")
    return {name.lower(): name for (name, *_) in cur.fetchall()}

def pick(colmap, candidates, required=True):
    """Pick the first candidate name that exists in the given column map.
    If required=False, return None when nothing matches instead of raising.
    """
    for c in candidates:
        k = c.lower()
        if k in colmap:
            return colmap[k]
    if required:
        raise ValueError(f"Missing any of {candidates}. Have: {sorted(colmap.values())}")
    return None

conn = mysql.connector.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=DB)
cur = conn.cursor()
cur.execute("SHOW TABLES")
tables = {t.lower(): t for (t,) in cur.fetchall()}

# Pick table names
orders = tables.get("orders")
orderdet = tables.get("order_details") or tables.get("orderdetails")
products = tables.get("products")
customers = tables.get("customers")  # optional for name later
if not (orders and orderdet and products):
    raise ValueError(f"Expected orders/order_details/products; have: {sorted(tables.values())}")

# Column maps (resolve exact column names safely)
o_cols = cols(cur, orders)
od_cols = cols(cur, orderdet)
p_cols = cols(cur, products)

# Keys and fields (case/variant tolerant)
# Identify join keys and important fields. The pick helper makes this robust
# to different naming conventions across Northwind variants.
o_id = pick(o_cols, ["orderid", "id", "order_id"])
o_date = pick(o_cols, ["orderdate", "order_date", "date"])
o_cust_id = pick(o_cols, ["customerid", "customer_id"], required=False)

od_order_id = pick(od_cols, ["orderid", "order_id"])
od_prod_id = pick(od_cols, ["productid", "product_id"])
od_price = pick(od_cols, ["unitprice", "unit_price", "price"])
od_qty = pick(od_cols, ["quantity", "qty"])
od_disc = pick(od_cols, ["discount", "discount_pct", "discount_rate"], required=False)

p_id = pick(p_cols, ["productid", "id", "product_id"])
p_name = pick(p_cols, ["productname", "product_name", "name"], required=False)

# Build SELECT fields
# We assemble a list of SQL fields and then join them with " ,\n  " for readability.
# line_total is computed as UnitPrice * Quantity * (1 - Discount). We use
# COALESCE to treat NULL discounts as 0. ProductName is included when present.
fields = [
    f"o.{o_id} AS OrderID",
    f"DATE(o.{o_date}) AS OrderDate",
]
if o_cust_id:
    fields.append(f"o.{o_cust_id} AS CustomerID")
else:
    fields.append("NULL AS CustomerID")

fields += [
    f"od.{od_prod_id} AS ProductID",
    f"od.{od_price} AS UnitPrice",
    f"od.{od_qty} AS Quantity",
]
if od_disc:
    fields.append(f"COALESCE(od.{od_disc}, 0) AS Discount")
    disc_expr = f"COALESCE(od.{od_disc}, 0)"
else:
    fields.append("0 AS Discount")
    disc_expr = "0"

if p_name:
    fields.append(f"p.{p_name} AS ProductName")

select_clause = ",\n  ".join(
    fields + [f"(od.{od_price} * od.{od_qty} * (1 - {disc_expr})) AS line_total"]
)

sql = f"""
SELECT
  {select_clause}
FROM {orders} o
JOIN {orderdet} od ON o.{o_id} = od.{od_order_id}
JOIN {products} p ON od.{od_prod_id} = p.{p_id}
"""

df = pd.read_sql(sql, conn)
conn.close()

os.makedirs("data/processed", exist_ok=True)
df.to_csv("data/processed/order_lines.csv", index=False)
len(df)


In [ ]:
import os
import pandas as pd

# KPIs (top customers and top products)
# Goal: load the extracted CSV, compute monthly revenue, Average Order Value (AOV),
# top customers, and top products. Save these to data/processed for reuse.

df = pd.read_csv("data/processed/order_lines.csv")
df["OrderDate"] = pd.to_datetime(df["OrderDate"], errors="coerce")
df = df.dropna(subset=["OrderDate"])

# Monthly revenue (sum of line totals grouped by calendar month). We group by
# Month Start (MS) so every month aligns to the first day of that month.
monthly = (
    df.groupby(pd.Grouper(key="OrderDate", freq="MS"))["line_total"]
      .sum().reset_index(name="revenue").sort_values("OrderDate")
)

# Average Order Value (AOV) per month: first sum each order’s lines, then take
# the mean across orders in that month.
orders = (
    df.groupby(["OrderID", pd.Grouper(key="OrderDate", freq="MS")])["line_total"]
      .sum().reset_index(name="order_value")
)
aov = orders.groupby("OrderDate")["order_value"].mean().reset_index(name="aov")

# Top 10 customers by revenue (include NaN customer IDs if present to surface
# anonymous/unknown cases).
top_customers = (
    df.groupby("CustomerID", dropna=False)["line_total"].sum()
      .sort_values(ascending=False).head(10).reset_index()
)

# Top products by revenue (use ProductName if available else ProductID)
by_product = df.groupby(
    (["ProductName"] if "ProductName" in df.columns else ["ProductID"])
)["line_total"].sum().sort_values(ascending=False).reset_index()
top_products = by_product.head(10)

os.makedirs("data/processed", exist_ok=True)
monthly.to_csv("data/processed/kpi_monthly_revenue.csv", index=False)
aov.to_csv("data/processed/kpi_aov_by_month.csv", index=False)
top_customers.to_csv("data/processed/kpi_top_customers.csv", index=False)
top_products.to_csv("data/processed/kpi_top_products.csv", index=False)

monthly.shape, aov.shape, top_customers.shape, top_products.shape


In [ ]:
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Currency/Thousands formatter for axes
currency = FuncFormatter(lambda x, pos: f"${x:,.0f}")

# Monthly revenue
plt.figure(figsize=(8,4))
plt.plot(monthly["OrderDate"], monthly["revenue"], marker="o")
plt.title("Monthly Revenue"); plt.xlabel("Month"); plt.ylabel("Revenue"); plt.tight_layout()
ax = plt.gca(); ax.yaxis.set_major_formatter(currency)
plt.savefig("figures/monthly_revenue.png", dpi=150); plt.show()

# Top customers
plt.figure(figsize=(8,4))
plt.barh(top_customers["CustomerID"].astype(str), top_customers["line_total"])
plt.title("Top 10 Customers by Revenue"); plt.xlabel("Revenue"); plt.tight_layout()
ax = plt.gca(); ax.xaxis.set_major_formatter(currency)
plt.savefig("figures/top_customers.png", dpi=150); plt.show()

# Top products
label_col = "ProductName" if "ProductName" in by_product.columns else "ProductID"
plt.figure(figsize=(8,4))
plt.barh(top_products[label_col].astype(str), top_products["line_total"])
plt.title("Top 10 Products by Revenue"); plt.xlabel("Revenue"); plt.tight_layout()
ax = plt.gca(); ax.xaxis.set_major_formatter(currency)
plt.savefig("figures/top_products.png", dpi=150); plt.show()

# Order value distribution
order_values = df.groupby("OrderID")["line_total"].sum().reset_index(name="order_value")
plt.figure(figsize=(8,4))
plt.hist(order_values["order_value"], bins=20, edgecolor="black")
plt.title("Order Value Distribution"); plt.xlabel("Order Value"); plt.ylabel("Count");
ax = plt.gca(); ax.xaxis.set_major_formatter(currency); plt.tight_layout()
plt.savefig("figures/order_value_distribution.png", dpi=150); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Monthly revenue + 3-month moving average
monthly_ma = monthly.copy()
monthly_ma["ma3"] = monthly_ma["revenue"].rolling(3, min_periods=1).mean()
plt.figure(figsize=(8,4))
plt.plot(monthly_ma["OrderDate"], monthly_ma["revenue"], marker="o", label="Revenue")
plt.plot(monthly_ma["OrderDate"], monthly_ma["ma3"], color="orange", label="3-mo MA")
plt.title("Monthly Revenue + 3-mo MA"); plt.xlabel("Month"); plt.ylabel("Revenue")
ax = plt.gca(); ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"${x:,.0f}"))
plt.legend(); plt.tight_layout()
plt.savefig("figures/monthly_revenue_ma.png", dpi=150); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Pareto chart: top 20 products (revenue + cumulative share)
label_col = "ProductName" if "ProductName" in by_product.columns else "ProductID"
bp = by_product.sort_values("line_total", ascending=False).copy()
bp["cum_share"] = bp["line_total"].cumsum() / bp["line_total"].sum() * 100
topn = bp.head(20).reset_index(drop=True)

fig, ax1 = plt.subplots(figsize=(9,4))
ax1.bar(range(len(topn)), topn["line_total"], color="#69b3a2")
ax1.set_ylabel("Revenue")
ax1.set_xticks(range(len(topn)))
ax1.set_xticklabels(topn[label_col].astype(str), rotation=45, ha="right")
ax1.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"${x:,.0f}"))

ax2 = ax1.twinx()
ax2.plot(range(len(topn)), topn["cum_share"], color="orange", marker="o")
ax2.set_ylabel("Cumulative %"); ax2.set_ylim(0, 110)

ax1.set_title("Pareto: Top 20 Products (Revenue + Cumulative Share)")
plt.tight_layout(); plt.savefig("figures/pareto_top_products.png", dpi=150); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Stacked area: monthly revenue split by top 5 products
label_col = "ProductName" if "ProductName" in by_product.columns else "ProductID"
top5 = by_product.head(5)[label_col].tolist()
stack = (df[df[label_col].isin(top5)]
         .groupby([pd.Grouper(key="OrderDate", freq="MS"), label_col])["line_total"]
         .sum().unstack(fill_value=0))
ax = stack.plot.area(figsize=(9,4), cmap="tab20")
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"${x:,.0f}"))
plt.title("Monthly Revenue — Top 5 Products"); plt.xlabel("Month"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig("figures/monthly_top5_products_area.png", dpi=150); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Revenue by weekday
wd = (df.assign(weekday=df["OrderDate"].dt.day_name())
          .groupby("weekday")["line_total"].sum())
order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
wd = wd.reindex(order).fillna(0)

plt.figure(figsize=(8,4))
plt.bar(wd.index, wd.values)
plt.title("Revenue by Weekday"); plt.xlabel("Weekday"); plt.ylabel("Revenue")
ax = plt.gca(); ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"${x:,.0f}"))
plt.xticks(rotation=30); plt.tight_layout()
plt.savefig("figures/revenue_by_weekday.png", dpi=150); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Simple 3-month baseline forecasts (naive and seasonal naive)
rev = monthly.sort_values("OrderDate").set_index("OrderDate")["revenue"]
future_periods = 3
future_idx = pd.date_range(rev.index[-1] + pd.offsets.MonthBegin(1), periods=future_periods, freq="MS")

# Naive: repeat last value
fc_naive = pd.Series(rev.iloc[-1], index=future_idx, name="Naive")

plt.figure(figsize=(9,4))
plt.plot(rev.index, rev.values, label="Revenue")
plt.plot(fc_naive.index, fc_naive.values, "--", label="Naive Forecast")

# Seasonal naive: same month last year (if enough data)
rev_m = rev.resample("MS").sum()
if len(rev_m) >= 12:
    last_year_idx = [t - pd.DateOffset(years=1) for t in future_idx]
    fc_seasonal = pd.Series(rev_m.reindex(last_year_idx).values, index=future_idx, name="Seasonal Naive")
    plt.plot(fc_seasonal.index, fc_seasonal.values, ":", label="Seasonal Naive")

plt.title("Monthly Revenue + Baseline Forecasts"); plt.xlabel("Month"); plt.ylabel("Revenue")
ax = plt.gca(); ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"${x:,.0f}"))
plt.legend(); plt.tight_layout(); plt.savefig("figures/revenue_forecasts.png", dpi=150); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("figures", exist_ok=True)

# Discount impact scatter
plt.figure(figsize=(8,4))
plt.scatter(df["Discount"], df["UnitPrice"] * df["Quantity"], s=20, alpha=0.6, edgecolors="none")
plt.title("Discount vs. Line Value"); plt.xlabel("Discount"); plt.ylabel("UnitPrice * Quantity")
ax = plt.gca(); ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"${x:,.0f}"))
plt.tight_layout(); plt.savefig("figures/discount_vs_linevalue.png", dpi=150); plt.show()
